## DATA PREPROCESSING

In [40]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix
import datetime as dt
from datetime import datetime

In [3]:
df = pd.read_excel("category.xlsx",sheet_name= [1,4])
loc_to_cat = df[1]
loc_to_cat['category1'] = loc_to_cat['category1'].str.lower()
allUserData = df[4]
#userData = [df[3], df[4], df[5], df[6], df[7]]

#for i in userData[1:]:
#    allUserData = allUserData.append(i)
allUserData.head()

,user_id,weekday,date,start_time,end_date,end_time,duration,category,category_id,location_name,location_category,my_place,location_id,place_id
0,2,2,2020-06-29,16:00:00,2020-06-29,19:00:00,03:00:00,meeting,13,路易莎,cafe,NaN,4.0,NaN
1,2,2,2020-06-29,19:00:00,2020-06-29,20:00:00,01:00:00,food,1,全哥鵝肉小炒,restaurant,NaN,5.0,NaN
2,2,2,2020-06-29,20:00:00,2020-06-29,20:30:00,00:30:00,leisure,10,NaN,NaN,home,NaN,1.0
3,2,2,2020-06-29,20:30:00,2020-06-29,21:30:00,01:00:00,shopping,12,Miramar,Shopping mall,NaN,6.0,NaN
4,2,2,2020-06-29,21:30:00,2020-06-29,23:30:00,02:00:00,study,5,NaN,NaN,home,NaN,1.0


In [4]:
allUserData['location']= allUserData['location_id'].combine_first(allUserData['place_id'])
#drop commute
indexNames = allUserData[(allUserData['category_id'] == 17) ].index
allUserData.drop(indexNames,inplace=True)

allUserData['location'].fillna("99", inplace = True)
allUserData = allUserData.drop(['date','end_date','end_time','location_name','location_category','my_place','location_id','place_id'], 1)
allUserData = allUserData.reset_index(drop = True)

In [5]:
def get_hours(ts):
    a = ts.hour
    return a

def get_mins(ts):
    a = ts.minute
    return a

def get_duration(ts):
    a = ts.hour*60 + ts.minute
    return a 

#allUserData['start_mins'] = allUserData['start_time'].apply(get_mins)
allUserData['start_time'] = allUserData['start_time'].apply(get_duration)
allUserData['duration'] = allUserData['duration'].apply(get_duration)
allUserData = allUserData[['user_id','weekday','start_time','duration','location','category','category_id']]
allUserData.tail()

,user_id,weekday,start_time,duration,location,category,category_id
444,2,5,50,495,1,sleep,3
445,2,5,545,15,1,leisure,10
446,2,5,570,150,2,work,6
447,2,5,780,360,3,meeting,13
448,2,5,1200,240,1,leisure,10


## Random Forest Model

In [6]:
train_df = allUserData.loc[0:299]
test_df = allUserData.loc[300:]
print("Number of traing data is :",len(train_df))
print("Number of testing data is :",len(test_df))

Number of traing data is : 300
Number of testing data is : 149


In [7]:
# without diff
X_train = train_df.iloc[:, np.r_[0:4]].values
y_train = train_df.iloc[:, 6].values

X_test = test_df.iloc[:, np.r_[0:4]].values
y_test = test_df.iloc[:, 6].values

In [8]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0,
            warm_start=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [9]:
clf.score(X_test, y_test)

0.6308724832214765

In [10]:
imp = clf.feature_importances_
imp

array([0.        , 0.11707604, 0.48461265, 0.3983113 ])

In [11]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.36      0.71      0.48        21
           3       1.00      0.95      0.97        19
           4       0.00      0.00      0.00         0
           6       0.50      0.82      0.62        11
           9       0.80      0.80      0.80         5
          10       0.80      0.71      0.75        58
          11       0.50      0.12      0.20         8
          12       0.44      0.25      0.32        16
          13       1.00      0.33      0.50         6
          16       0.00      0.00      0.00         2
          18       0.00      0.00      0.00         3

   micro avg       0.63      0.63      0.63       149
   macro avg       0.49      0.43      0.42       149
weighted avg       0.67      0.63      0.62       149



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [12]:
print(confusion_matrix(y_test,predictions))

[[15  0  0  0  0  4  1  1  0  0  0]
 [ 0 18  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  1  0  0  0  1  0]
 [ 0  0  0  0  4  1  0  0  0  0  0]
 [ 8  0  1  5  0 41  0  3  0  0  0]
 [ 5  0  0  0  0  2  1  0  0  0  0]
 [10  0  0  0  1  1  0  4  0  0  0]
 [ 0  0  0  4  0  0  0  0  2  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  1  0  0  0]]


In [13]:
predictions

array([12,  1,  1, 10, 10, 10,  3, 10, 10,  1, 10, 11,  1,  1,  4, 10,  3,
       10,  6, 10, 10, 10, 10,  1,  6,  1, 12, 10, 10,  3,  6,  6, 10,  9,
       10,  3,  6,  1,  1, 13, 10, 10,  9, 10,  3,  6,  6,  1, 10, 10,  3,
       10,  1,  1, 12,  1,  1, 11,  1, 10,  3, 10,  3, 10, 12, 12,  1,  6,
        1,  1, 12,  1, 10,  3, 10,  6,  1, 10, 12,  9, 10,  3,  6,  6,  1,
       10,  9, 10, 10,  3, 10,  6,  1, 13,  1,  1,  1, 10, 10,  3, 10,  1,
       16,  1, 10,  3, 10,  1,  1, 10,  1,  1,  1, 10,  3, 10,  1, 10, 12,
       10,  3, 10,  6,  1, 12, 10,  3, 10,  1,  6,  1, 10,  3,  6,  6,  1,
        1,  1,  1,  1,  1, 10,  9, 10,  3, 10,  6,  6, 10])

## User Data

In [142]:
location = pd.read_csv("hsieh.csv")
location.tail()

,location_id,user_id,longitude,latitude,start_date,start_time,duration,weekday,name1,name2,name3,name4,name5,category1,category2,category3,category4,category5
290,291,1,120.456609,23.445741,2020-10-01,16:44,NaN,5,長吉本舖,素齋麵食館(素食),全家(中埔中華店),芭那那早餐,Sanhe Park,Bakery,Restaurant,Convenience store,Restaurant,Park
291,292,1,120.456894,23.445873,2020-10-01,19:15,NaN,5,芭那那早餐,素齋麵食館(素食),全家(中埔中華店),長吉本舖,Sanhe Park,Restaurant,Restaurant,Convenience store,Bakery,Park
292,293,1,120.456551,23.445965,2020-10-01,19:29,NaN,5,素齋麵食館(素食),芭那那早餐,嘉陽電腦剌繡,全家(中埔中華店),長吉本舖,Restaurant,Restaurant,Embroidery service,Convenience store,Bakery
293,294,1,120.456822,23.445649,2020-10-01,19:43,NaN,5,芭那那早餐,素齋麵食館(素食),嘉陽電腦剌繡,平實乙村,Sanhe Park,Restaurant,Restaurant,Embroidery service,Townhouse complex,Park
294,295,1,120.456809,23.445631,2020-10-01,21:56,NaN,5,芭那那早餐,素齋麵食館(素食),平實乙村,Sanhe Park,長吉本舖,Restaurant,Restaurant,Townhouse complex,Park,Townhouse complex


In [143]:
stopLoc = location.drop(['category3','category4','category5'], 1)

# def string_to_mins(ts):
#     a = int(ts.split(':')[0])
#     b = int(ts.split(':')[1])
#     c = a*60 + b
#     return c

#stopLoc['start_time'] = pd.to_datetime(stopLoc['start_time'], format='%H:%M', errors='ignore').apply(string_to_mins)
stopLoc['longitude'] = round(stopLoc['longitude'],2)
stopLoc['latitude'] = round(stopLoc['latitude'],3)
stopLoc = stopLoc.loc[stopLoc['name1'] != 'nccu dorm9']
stopLoc = stopLoc.reset_index(drop = True) 

In [144]:
def deleteEnter(x):
    enterList = []
    dropList = []

    for a in range(len(stopLoc)):

        #如果這筆是「enter」，記住編號
        if (x.iloc[a,14] == 'entering myPlace'):
            enterList.append(a)

        #如果這筆是「exit」，清空enterList
        elif (x.iloc[a,14] == 'exiting myPlace'):
            enterList = []

        elif (enterList == []):
            pass

        #如果這筆不是進去也不是出來，且有enterList，為要刪除的東西
        elif (enterList != []):
            dropList.append(a) 

    x = x.drop(dropList)
    x = x.reset_index(drop = True)  
    return(x)

stopLoc = deleteEnter(stopLoc)
stopLoc

,location_id,user_id,longitude,latitude,start_date,start_time,duration,weekday,name1,name2,name3,name4,name5,category1,category2
0,1,1,121.47,25.023,2020-09-21,21:14,NaN,2,HOME,1,NaN,NaN,NaN,Home,entering myPlace
1,14,1,121.47,25.023,2020-09-22,11:25,NaN,3,HOME,1,NaN,NaN,NaN,Home,exiting myPlace
2,15,1,121.47,25.022,2020-09-22,11:26,NaN,3,Oyami Cafe 新埔店,板橋花市,創價學會板橋藝文中心,爭鮮迴轉壽司-新埔店,全家便利商店 板橋文興店,restaurant,point_of_interest
3,16,1,121.54,25.041,2020-09-22,12:02,NaN,3,鼎泰豐 復興店,頂呱呱復興店,City’Super 復興SOGO店,成真咖啡台北sogo復興店,點水樓(復興店),restaurant,restaurant
4,17,1,121.54,25.041,2020-09-22,12:04,NaN,3,台北福華大飯店The Howard Plaza Hotel Taipei,丹堤咖啡,Plants,葛鎂診所,花園大道(自助餐) Park Avenue-台北福華大飯店,lodging,cafe
5,18,1,121.56,25.004,2020-09-22,12:12,NaN,3,7-ELEVEn 辛亥站門市,滑遍天下滑雪小舖(沒有夏天),信義房屋 捷運辛亥店,環遊市,滑雪小舖,convenience_store,point_of_interest
6,19,1,121.58,24.998,2020-09-22,12:14,NaN,3,動物園站,臺北市立動物園,木柵動物園停車場（河川高灘地）,葡而飛街角複合式簡餐,臺北市立動物園臺灣動物區,subway_station,zoo
7,20,1,121.58,24.990,2020-09-22,12:20,NaN,3,Taibeishiwenshanquwanxing Elementary School,萬興國小運動中心-恆動力萬興營業所,Da Cheng High School,Dacheng High School,神奇山莊社區,primary_school,point_of_interest
8,21,1,121.58,24.987,2020-09-22,12:22,NaN,3,NCCU,2,NaN,NaN,NaN,School,entering myPlace
9,28,1,121.58,24.987,2020-09-22,17:55,NaN,3,DORM,2,NaN,NaN,NaN,Dorm,entering myPlace


In [145]:
for row in range(0,len(stopLoc)-1): 

    #calculate duration
    if stopLoc.iloc[row+1,4] == stopLoc.iloc[row,4]:
        stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,5], '%H:%M') - datetime.strptime(stopLoc.iloc[row,5], '%H:%M')       
    else:
        a = dt.datetime(2000,1,2,int(stopLoc.iloc[row+1,5].split(':')[0]),int(stopLoc.iloc[row+1,5].split(':')[1]))
        b = dt.datetime(2000,1,1,int(stopLoc.iloc[row,5].split(':')[0]),int(stopLoc.iloc[row,5].split(':')[1]))
        stopLoc.iloc[row,6] = a - b

#delete trivial data, 重複地點太多刪除
dropList = []
for row in range(1,len(stopLoc)-1): 
    if stopLoc.iloc[row,14] != 'entering myPlace' and stopLoc.iloc[row,14] != 'exiting myPlace':
        
        #same longitude and lantitude
        #if stopLoc.iloc[row,2] == stopLoc.iloc[row-1,2] and stopLoc.iloc[row,3] == stopLoc.iloc[row-1,3]:
        #    dropList.append(row)

        #compared with the last record, over 2 place name duplicated will be deleted
        res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
        #if stopLoc.iloc[row-1,6] < pd.Timedelta('00:10:00'):
        print(res)
        if len(res) > 1: 
            dropList.append(row)

stopLoc = stopLoc.drop(dropList)
stopLoc = stopLoc.reset_index(drop = True)
stopLoc

set()
set()
set()
set()
set()
set()
set()
{'NCCU College of Commerce', '政大四維堂', 'National Chengchi University', '政大行政大樓'}
{'NCCU College of Commerce', '政大四維堂', 'National Chengchi University', '政大行政大樓'}
set()
{'NCCU College of Commerce', '國立政治大學商學院圖書分館', '萊爾富便利商店'}
{'國立政治大學商學院圖書分館', '萊爾富便利商店'}
{'國立政治大學圖書館', '國立政治大學商學院', '萊爾富便利商店', '國立政治大學商學院圖書分館', '國立政治大學'}
{'國立政治大學商學院', '國立政治大學'}
set()
set()
{'國立政治大學商學院圖書分館', '萊爾富便利商店'}
{'國立政治大學商學院圖書分館', '國立政治大學圖書館', '國立政治大學商學院', '萊爾富便利商店'}
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
{'臺灣法國文化協會(臺北)', '國立臺灣大學 進修推廣學院', 'Taiwan Power Research Institute', '百老匯影城'}
set()
set()
set()
set()
set()
set()
set()
set()
{'National Chengchi University', '金鮨日式料理', '滇味廚房'}
{'大汗麻辣鴨血臭豆腐(政大店)', '7-Eleven', '滇味廚房'}
{'大汗麻辣鴨血臭豆腐(政大店)', '7-Eleven', '滇味廚房', '提洛斯義式廚房'}
{'大汗麻辣鴨血臭豆腐(政大店)', 'LAZY PASTA 慵懶義式廚房 政大店', '滇味廚房', '7-Eleven', '提洛斯義式廚房'}
{'大汗麻辣鴨血臭豆腐(政大店)', 'LAZY PASTA 慵懶義式廚房 政大店', '滇味廚房', '7-Eleven', '提洛斯義式廚房'}
set()
set()
{'國立政治大學圖書館', '國立政治大學商學院', 

,location_id,user_id,longitude,latitude,start_date,start_time,duration,weekday,name1,name2,name3,name4,name5,category1,category2
0,1,1,121.47,25.023,2020-09-21,21:14,14:11:00,2,HOME,1,NaN,NaN,NaN,Home,entering myPlace
1,14,1,121.47,25.023,2020-09-22,11:25,0:01:00,3,HOME,1,NaN,NaN,NaN,Home,exiting myPlace
2,15,1,121.47,25.022,2020-09-22,11:26,0:36:00,3,Oyami Cafe 新埔店,板橋花市,創價學會板橋藝文中心,爭鮮迴轉壽司-新埔店,全家便利商店 板橋文興店,restaurant,point_of_interest
3,16,1,121.54,25.041,2020-09-22,12:02,0:02:00,3,鼎泰豐 復興店,頂呱呱復興店,City’Super 復興SOGO店,成真咖啡台北sogo復興店,點水樓(復興店),restaurant,restaurant
4,17,1,121.54,25.041,2020-09-22,12:04,0:08:00,3,台北福華大飯店The Howard Plaza Hotel Taipei,丹堤咖啡,Plants,葛鎂診所,花園大道(自助餐) Park Avenue-台北福華大飯店,lodging,cafe
5,18,1,121.56,25.004,2020-09-22,12:12,0:02:00,3,7-ELEVEn 辛亥站門市,滑遍天下滑雪小舖(沒有夏天),信義房屋 捷運辛亥店,環遊市,滑雪小舖,convenience_store,point_of_interest
6,19,1,121.58,24.998,2020-09-22,12:14,0:06:00,3,動物園站,臺北市立動物園,木柵動物園停車場（河川高灘地）,葡而飛街角複合式簡餐,臺北市立動物園臺灣動物區,subway_station,zoo
7,20,1,121.58,24.990,2020-09-22,12:20,0:02:00,3,Taibeishiwenshanquwanxing Elementary School,萬興國小運動中心-恆動力萬興營業所,Da Cheng High School,Dacheng High School,神奇山莊社區,primary_school,point_of_interest
8,21,1,121.58,24.987,2020-09-22,12:22,5:33:00,3,NCCU,2,NaN,NaN,NaN,School,entering myPlace
9,28,1,121.58,24.987,2020-09-22,17:55,3:01:00,3,DORM,2,NaN,NaN,NaN,Dorm,entering myPlace


In [134]:
pd.set_option('display.max_rows', 5000)

In [135]:
#kill duration < 10 mins, and keep the start of commuting

dropList = []
for row in range(0,len(stopLoc)-1): 

    #calculate duration
    if stopLoc.iloc[row+1,4] == stopLoc.iloc[row,4]:
        stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,5], '%H:%M') - datetime.strptime(stopLoc.iloc[row,5], '%H:%M')       
    else:
        a = dt.datetime(2000,1,2,int(stopLoc.iloc[row+1,5].split(':')[0]),int(stopLoc.iloc[row+1,5].split(':')[1]))
        b = dt.datetime(2000,1,1,int(stopLoc.iloc[row,5].split(':')[0]),int(stopLoc.iloc[row,5].split(':')[1]))
        stopLoc.iloc[row,6] = a - b
            
    #kill duration <10
    if pd.Timedelta(stopLoc.iloc[row,6]) < pd.Timedelta('00:10:00'):
        if stopLoc.iloc[row,14] != 'entering myPlace' and stopLoc.iloc[row,14] != 'exiting myPlace':
            dropList.append(row)    
            
removeList = []
for i in range(1,len(dropList)-1):
    if dropList[i] - dropList[i-1] < 2:
        removeList.append(dropList[i])

stopLoc = stopLoc.drop(removeList)
stopLoc = stopLoc.reset_index(drop = True)
stopLoc

,location_id,user_id,longitude,latitude,start_date,start_time,duration,weekday,name1,name2,name3,name4,name5,category1,category2
0,1,1,121.47,25.023,2020-09-21,21:14,14:11:00,2,HOME,1,NaN,NaN,NaN,Home,entering myPlace
1,14,1,121.47,25.023,2020-09-22,11:25,0:01:00,3,HOME,1,NaN,NaN,NaN,Home,exiting myPlace
2,15,1,121.47,25.022,2020-09-22,11:26,0:36:00,3,Oyami Cafe 新埔店,板橋花市,創價學會板橋藝文中心,爭鮮迴轉壽司-新埔店,全家便利商店 板橋文興店,restaurant,point_of_interest
3,16,1,121.54,25.041,2020-09-22,12:02,0:10:00,3,鼎泰豐 復興店,頂呱呱復興店,City’Super 復興SOGO店,成真咖啡台北sogo復興店,點水樓(復興店),restaurant,restaurant
4,18,1,121.56,25.004,2020-09-22,12:12,0:02:00,3,7-ELEVEn 辛亥站門市,滑遍天下滑雪小舖(沒有夏天),信義房屋 捷運辛亥店,環遊市,滑雪小舖,convenience_store,point_of_interest
5,21,1,121.58,24.987,2020-09-22,12:22,5:33:00,3,NCCU,2,NaN,NaN,NaN,School,entering myPlace
6,28,1,121.58,24.987,2020-09-22,17:55,3:01:00,3,DORM,2,NaN,NaN,NaN,Dorm,entering myPlace
7,31,1,121.58,24.987,2020-09-22,20:56,1:19:00,3,DORM,2,NaN,NaN,NaN,Dorm,exiting myPlace
8,35,1,121.58,24.987,2020-09-22,22:15,10:48:00,3,DORM,2,NaN,NaN,NaN,Dorm,entering myPlace
9,42,1,121.58,24.987,2020-09-23,9:03,3:12:00,4,DORM,2,NaN,NaN,NaN,Dorm,exiting myPlace


In [136]:
#delete duplicate:重複進出同地點, 刪時間短的,重算時間
duplicate = []
for row in range(0,len(stopLoc)-1): 
    
    if (stopLoc.iloc[row,14] == 'exiting myPlace') and (stopLoc.iloc[row+2,14] == 'entering myPlace'):
        if (stopLoc.iloc[row,8] == stopLoc.iloc[row+2,8]):
            duplicate.extend([row,row+1,row+2])
    
    #kill duration <10
    if pd.Timedelta(stopLoc.iloc[row,6]) < pd.Timedelta('00:10:00'):
        if stopLoc.iloc[row,14] != 'entering myPlace' and stopLoc.iloc[row,14] != 'exiting myPlace':
            duplicate.append(row) 

stopLoc = stopLoc.drop(duplicate)
stopLoc = stopLoc.reset_index(drop = True)
            
for row in range(0,len(stopLoc)-1): 
    #calculate duration
    if stopLoc.iloc[row+1,4] == stopLoc.iloc[row,4]:
        stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,5], '%H:%M') - datetime.strptime(stopLoc.iloc[row,5], '%H:%M')       
    else:
        a = dt.datetime(2000,1,2,int(stopLoc.iloc[row+1,5].split(':')[0]),int(stopLoc.iloc[row+1,5].split(':')[1]))
        b = dt.datetime(2000,1,1,int(stopLoc.iloc[row,5].split(':')[0]),int(stopLoc.iloc[row,5].split(':')[1]))
        stopLoc.iloc[row,6] = a - b

stopLoc

,location_id,user_id,longitude,latitude,start_date,start_time,duration,weekday,name1,name2,name3,name4,name5,category1,category2
0,1,1,121.47,25.023,2020-09-21,21:14,14:11:00,2,HOME,1,NaN,NaN,NaN,Home,entering myPlace
1,14,1,121.47,25.023,2020-09-22,11:25,0:01:00,3,HOME,1,NaN,NaN,NaN,Home,exiting myPlace
2,15,1,121.47,25.022,2020-09-22,11:26,0:36:00,3,Oyami Cafe 新埔店,板橋花市,創價學會板橋藝文中心,爭鮮迴轉壽司-新埔店,全家便利商店 板橋文興店,restaurant,point_of_interest
3,16,1,121.54,25.041,2020-09-22,12:02,0:20:00,3,鼎泰豐 復興店,頂呱呱復興店,City’Super 復興SOGO店,成真咖啡台北sogo復興店,點水樓(復興店),restaurant,restaurant
4,21,1,121.58,24.987,2020-09-22,12:22,5:33:00,3,NCCU,2,NaN,NaN,NaN,School,entering myPlace
5,28,1,121.58,24.987,2020-09-22,17:55,3:01:00,3,DORM,2,NaN,NaN,NaN,Dorm,entering myPlace
6,31,1,121.58,24.987,2020-09-22,20:56,1:19:00,3,DORM,2,NaN,NaN,NaN,Dorm,exiting myPlace
7,35,1,121.58,24.987,2020-09-22,22:15,10:48:00,3,DORM,2,NaN,NaN,NaN,Dorm,entering myPlace
8,42,1,121.58,24.987,2020-09-23,9:03,3:12:00,4,DORM,2,NaN,NaN,NaN,Dorm,exiting myPlace
9,48,1,121.57,24.987,2020-09-23,12:15,1:14:00,4,H.I.Feeling Coffee早午餐,麥當勞-台北指南店,波波恰恰大馬風味餐,鬼匠拉麵-政大店,萊爾富便利商店 北市指南店,cafe,restaurant


In [137]:
#delete trivial data, 重複地點太多刪除

dropList = []
for row in range(1,len(stopLoc)-1): 
    if stopLoc.iloc[row,14] != 'entering myPlace' and stopLoc.iloc[row,14] != 'exiting myPlace':
        
        #same longitude and lantitude
        if stopLoc.iloc[row,1] == stopLoc.iloc[row-1,1] and stopLoc.iloc[row,2] == stopLoc.iloc[row-1,2]:
            dropList.append(row)

        #compared with the last record, over 2 place name duplicated will be deleted
        res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
        if len(res) > 1: 
            dropList.append(row)

stopLoc = stopLoc.drop(dropList)
stopLoc = stopLoc.drop(['name3','name4','name5'], 1)
stopLoc = stopLoc.reset_index(drop = True)

for row in range(0,len(stopLoc)-1): 
    #calculate duration
    if stopLoc.iloc[row+1,4] == stopLoc.iloc[row,4]:
        stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,5], '%H:%M') - datetime.strptime(stopLoc.iloc[row,5], '%H:%M')       
    else:
        a = dt.datetime(2000,1,2,int(stopLoc.iloc[row+1,5].split(':')[0]),int(stopLoc.iloc[row+1,5].split(':')[1]))
        b = dt.datetime(2000,1,1,int(stopLoc.iloc[row,5].split(':')[0]),int(stopLoc.iloc[row,5].split(':')[1]))
        stopLoc.iloc[row,6] = a - b
stopLoc

,location_id,user_id,longitude,latitude,start_date,start_time,duration,weekday,name1,name2,category1,category2
0,1,1,121.47,25.023,2020-09-21,21:14,14:11:00,2,HOME,1,Home,entering myPlace
1,14,1,121.47,25.023,2020-09-22,11:25,0:37:00,3,HOME,1,Home,exiting myPlace
2,16,1,121.54,25.041,2020-09-22,12:02,0:20:00,3,鼎泰豐 復興店,頂呱呱復興店,restaurant,restaurant
3,21,1,121.58,24.987,2020-09-22,12:22,5:33:00,3,NCCU,2,School,entering myPlace
4,28,1,121.58,24.987,2020-09-22,17:55,3:01:00,3,DORM,2,Dorm,entering myPlace
5,31,1,121.58,24.987,2020-09-22,20:56,1:19:00,3,DORM,2,Dorm,exiting myPlace
6,35,1,121.58,24.987,2020-09-22,22:15,10:48:00,3,DORM,2,Dorm,entering myPlace
7,42,1,121.58,24.987,2020-09-23,9:03,3:12:00,4,DORM,2,Dorm,exiting myPlace
8,48,1,121.57,24.987,2020-09-23,12:15,1:14:00,4,H.I.Feeling Coffee早午餐,麥當勞-台北指南店,cafe,restaurant
9,49,1,121.58,24.986,2020-09-23,13:29,5:07:00,4,National Chengchi University,NCCU College of Commerce,university,university


In [138]:
#標commute
twtymin = pd.Timedelta('00:20:00')

for row in range(0,len(stopLoc)-1): 

    if stopLoc.iloc[row,11] != 'entering myPlace':
        #mark as commute if distance bigger than 500 meters. 
        if (abs(stopLoc.iloc[row+1,2] - stopLoc.iloc[row,2]) + abs(stopLoc.iloc[row+1,3] - stopLoc.iloc[row,3])) > 0.04:
            stopLoc.iloc[row,10] = 'commute'
            
#         elif (abs(stopLoc.iloc[row-1,1] - stopLoc.iloc[row,1]) + abs(stopLoc.iloc[row-1,2] - stopLoc.iloc[row,2])) > 0.02:
#             stopLoc.iloc[row,7] = 'commute'
            
        if stopLoc.iloc[row,11] == 'exiting myPlace':
            stopLoc.iloc[row,10] = 'commute' 
            
        if pd.Timedelta(stopLoc.iloc[row-1,6]) > twtymin :
            if abs(stopLoc.iloc[row-1,2] - stopLoc.iloc[row,2]) > 0.02 or abs(stopLoc.iloc[row-1,3] - stopLoc.iloc[row,3]) > 0.02:
                stopLoc.iloc[row,10] = 'commute'        
        
stopLoc

,location_id,user_id,longitude,latitude,start_date,start_time,duration,weekday,name1,name2,category1,category2
0,1,1,121.47,25.023,2020-09-21,21:14,14:11:00,2,HOME,1,Home,entering myPlace
1,14,1,121.47,25.023,2020-09-22,11:25,0:37:00,3,HOME,1,commute,exiting myPlace
2,16,1,121.54,25.041,2020-09-22,12:02,0:20:00,3,鼎泰豐 復興店,頂呱呱復興店,commute,restaurant
3,21,1,121.58,24.987,2020-09-22,12:22,5:33:00,3,NCCU,2,School,entering myPlace
4,28,1,121.58,24.987,2020-09-22,17:55,3:01:00,3,DORM,2,Dorm,entering myPlace
5,31,1,121.58,24.987,2020-09-22,20:56,1:19:00,3,DORM,2,commute,exiting myPlace
6,35,1,121.58,24.987,2020-09-22,22:15,10:48:00,3,DORM,2,Dorm,entering myPlace
7,42,1,121.58,24.987,2020-09-23,9:03,3:12:00,4,DORM,2,commute,exiting myPlace
8,48,1,121.57,24.987,2020-09-23,12:15,1:14:00,4,H.I.Feeling Coffee早午餐,麥當勞-台北指南店,cafe,restaurant
9,49,1,121.58,24.986,2020-09-23,13:29,5:07:00,4,National Chengchi University,NCCU College of Commerce,commute,university


In [102]:
stopLoc = stopLoc.drop(['location_id','user_id','longitude','latitude'], 1)
stopLoc

,start_date,start_time,duration,weekday,name1,name2,category1,category2
0,2020-08-20,13:01,0:21:00,5,Mai Hsiang Yuen Handmade Pizza,10NT$ Self Serve Car Wash,restaurant,car_wash
1,2020-08-20,13:22,4:52:00,5,米塔義式廚房-政大店,Louisa Coffee,commute,cafe
2,2020-08-20,18:14,3:38:00,5,Xiangshan Station,WuPaoChun Bakery Taipei,commute,bakery
3,2020-08-20,21:52,0:28:00,5,陸光小館,永固便利停車,commute,parking
4,2020-08-20,22:20,3:37:00,5,Carrefour DaZhi Store,Miramar Ferris Wheel,commute,point_of_interest
5,2020-08-21,1:57,9:53:00,6,漁網居酒屋,中山公園,commute,park
6,2020-08-21,11:50,1:25:00,6,阿華食堂,軟訊資訊有限公司,commute,electronics_store
7,2020-08-21,13:15,2:49:00,6,一直咖啡 Izhi Coffee,好奇營養豆花 大直店,commute,food
8,2020-08-21,16:04,1:42:00,6,臺北市私立怡寶托嬰中心,甫月fuyue,commute,point_of_interest
9,2020-08-21,17:46,0:15:00,6,Shandao Temple Station,McDonald's,commute,restaurant


## Map Category

In [ ]:
stopLoc = pd.merge(stopLoc, loc_to_cat, how='left', on='category1' )
stopLoc = stopLoc.drop(['name1','category1','location_id'],1)
stopLoc['user_id'] = 2
stopLoc = stopLoc[['user_id','weekday','start_time','duration','location']]
stopLoc
#把start_date加回來
#name1,category1拿掉 place_cat對表換成數字

In [ ]:
traing = stopLoc.iloc[:, np.r_[0:4]].values
predictions = clf.predict(traing)

## Myplace

In [ ]:
savedplace = pd.read_csv("savedPlace.csv")
savedplace['place_longitude'] = round(saveplace['place_longitude'],3)
savedplace['place_latitude'] = round(saveplace['place_latitude'],3)
#savedplace = savedplace.loc[savedplace['my_place'] == 1]
savedplace = savedplace.drop(['place_name','my_place','place_id'],1)
savedplace = savedplace.rename(columns={"place_longitude": "longitude", "place_latitude": "latitude"})
savedplace = savedplace.drop_duplicates(subset=['longitude', 'latitude'], keep='last')
savedplace.head()

In [ ]:
stopLoc = pd.merge(stopLoc, savedplace, how='left', on=['longitude','latitude'])
stopLoc